In [ ]:
!pip install transformers
!pip install pytorch-lightning

In [ ]:
import numpy as np
import pandas as pd

import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.core.lightning import LightningModule
from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
import math
import random
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
PATH = "/content/drive/MyDrive/쿠빅/nlp챗봇/"
      
a = '정환'
b = '택'
c = '선우'
d = '동룡'

## (1) 데이터 전처리

In [ ]:
df_a = pd.read_csv("/content/drive/MyDrive/쿠빅/nlp챗봇/"+a+'.csv', encoding ='cp949')
df_b = pd.read_csv("/content/drive/MyDrive/쿠빅/nlp챗봇/"+b+'.csv', encoding ='cp949')
df_c = pd.read_csv("/content/drive/MyDrive/쿠빅/nlp챗봇/"+c+'.csv', encoding ='cp949')
df_d = pd.read_csv("/content/drive/MyDrive/쿠빅/nlp챗봇/"+d+'.csv', encoding ='cp949')

In [ ]:
df_a

,Q,A,role
0,정환아..너한테 할 말 있어서..,"진짜 아, 이런 것 좀 하지 마세요.",정환
1,정환아..너한테 할 말 있어서..,"아, 이런 진짜 것 좀 하지 마세요.",정환
2,정환아..너한테 할 말 있어서..,"아, 진짜 것 좀 하지 마세요.",정환
3,정환아..너한테 할 말 있어서..,"아, 진짜 이런 것 좀 하지 마세요.",정환
4,야. 정환아.,"진짜 아, 이런 것 좀 하지 마세요.",정환
...,...,...,...
17224,야 차. 사천 내려가야하잖아.,차 고장났어. 버스타고 먹고 가려고. 푹 자고 가지,정환
17225,야 차. 사천 내려가야하잖아.,차 고장났어. 버스타고 가려고. 먹고 푹 자고 가지,정환
17226,야 너 차. 사천 내려가야하잖아.,버스타고 차 고장났어. 가려고. 먹고 푹 자고 가지,정환
17227,야 너 차. 사천 내려가야하잖아.,차 고장났어. 버스타고 먹고 가려고. 푹 자고 가지,정환


필요없는 열 삭제

In [ ]:
df=df_d

In [ ]:
df.drop([ 'role'], inplace = True, axis = 1)

## (2) 모델링
GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = "<usr>"
        self.a_token = "<sys>"
        self.bos = '</s>'
        self.eos = '</s>'
        self.pad = '<pad>'
        self.mask = '<mask>'
        self.tokenizer = tokenizer

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문 열을 가져온다.
        a = turn["A"]  # 답변 열을 가져온다.

        q_toked = self.tokenizer.tokenize(self.q_token + q  + self.bos)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        # 최대길이 - 질문길이
            if a_len <= 0:       # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   # 질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로 
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.    
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).to(device), torch.LongTensor(mask).to(device), torch.LongTensor(label).to(device)

In [ ]:
train_set = ChatbotDataset(df, max_len=40) 
train_dataloader = DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch,)

In [ ]:
model  = model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [ ]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction="none")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 5
Sneg = -1e18

In [ ]:
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        out = model(token_ids)
        out = out.logits      #Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print ("end")

100%|██████████| 5/5 [16:20<00:00, 196.07s/it]

end


In [ ]:
torch.save(model, "/content/drive/MyDrive/쿠빅/nlp챗봇/동룡.pt")

## 챗봇 결과


In [ ]:
import numpy as np
import pandas as pd

import torch

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                                                    bos_token='</s>', eos_token='</s>', unk_token='<unk>',pad_token='<pad>', mask_token='<mask>')

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [ ]:
PATH = "/content/drive/MyDrive/쿠빅/nlp챗봇/"
      
a = '정환'
b = '택'
c = '선우'
d = '동룡'

jh_model = torch.load(PATH + a +'.pt', map_location=device)
t_model = torch.load(PATH + b +'.pt', map_location=device)
sw_model = torch.load(PATH + c +'.pt', map_location=device)
dr_model = torch.load(PATH + d +'.pt', map_location=device)

In [ ]:
with torch.no_grad():
    while 1:
      who = input("두근두근 소개팅 대화 상대를 골라주세요 >< > \n(정환,택,선우,동룡 중 1명 입력)").strip()
      if who =='정환':
        model = jh_model
        break
      if who =='택':
        model = t_model
        break
      if who =='선우':
        model = sw_model
        break
      if who =='동룡':
        model = dr_model
        break
      else : 
        who = input("이 중에서는 맘에 드는 상대가 없으신가요? ㅠㅠ > \n(정환,택,선우,동룡 중 1명 입력)").strip()

    user = input("본인 이름을 입력해주세요! > ").strip()
    print("지금부터 "+who+"의 소개팅이 시작됩니다. 두근두근!")

    print("끼이익 또각또각 쿵.....")
    


    print(who+" > 안녕 {}!".format(user.strip()))

    while 1:
        
        q = input("User > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(tokenizer.encode("<usr>" + q +"<sys>" + a)).unsqueeze(dim=0)
            input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = tokenizer.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == '</s>':
                break
            a += gen.replace("▁", " ")
        print(who+" > {}".format(a.strip()))

지금부터 택의 소개팅이 시작됩니다. 두근두근!
끼이익 또각또각 쿵.....
택 > 안녕 지호!
